### Part 1: AWS S3 & Sourcing Datasets
Republish this open dataset in Amazon S3 and share with us a link.
You may run into 403 Forbidden errors as you test accessing this data. There is a way to comply with the BLS data access policies and re-gain access to fetch this data programatically - we have included some hints as to how to do this at the bottom of this README in the Q/A section.

# Required sub-steps to accomplisht this:
1. Set up S3 bucket environmnet
2. Be able to read and publish into S3 env. 
3. troubleshoot error

- Key notes: This is DataLake design. Create a landing-zone where data will be uploaded and parsed under upload date. The goal is just to have a starting point to being bringing in data into s3.

In [ ]:
#Create s3 wrapper class

